# Pipeline de extracción, este es el bueno

In [1]:
import fitz
import pandas as pd
import nltk
import os 

nltk.download('cess_esp')
from nltk.corpus import cess_esp
#Por mejorar, este corpus está chiquito
spanish_words = set(word for word in cess_esp.words())

path = "data/"
documents = []

# Get all the PDF documents NAMES from the path
for file in os.listdir(path):
    if file.endswith(".pdf"):
        documents.append(file[:-4])


[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\Maracuya\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


In [3]:
import re


def get_text_from_pdf(key):
    doc = fitz.open(f'data/{key}.pdf')
    number_of_pages = doc.page_count

    i=documents[key][0]
    full_text=""
    for page in doc.pages(documents[key][0], number_of_pages-documents[key][1]):
        # print("Page number: ", i)
        this_page_text=page.get_text("text", sort=True).replace("\n", " ")
        this_page_text=re.sub(r'\d+', '', this_page_text)
        full_text+=this_page_text+" "
        i+=1
    return full_text

def get_all_text_from_pdf(key):
    doc = fitz.open(f'data/{key}.pdf')
    full_text=" ".join([page.get_text("text", sort=True).replace("\n", " ") for page in doc])
    full_text=re.sub(r'\d+', '', full_text)
    return full_text

In [4]:
from nltk.tokenize import sent_tokenize
from langdetect import detect
import langid

def filtrar_espaniol(full_text):
    sentences = sent_tokenize(full_text)
    non_spanish_sentences = []

    
    for sentence in sentences:
        try:
            languages=[]
            languages.append(detect(sentence))
            languages.append(langid.classify(sentence)[0])
            if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
                non_spanish_sentences.append(sentence)
        except:
            pass

    sentences = non_spanish_sentences
    

    return sentences

def filtrar_oracion_espaniol(sentence):
    try:
        languages=[]
        languages.append(detect(sentence))
        languages.append(langid.classify(sentence)[0])
        if not any(lang in languages for lang in ["es", "it", "ca", "pt"]):
            return True
    except:
        pass

    return False

In [5]:
#Funciones de filtrado
alfabeto_quechua = ['a', 'aa', 'ch', 'chh', 'ch\'', 'ts', 'tr', 'h', 'i', 'ii', 'k', 'kh', 'k\'', 'l', 'll', 'm', 'n', 'ñ', 'p', 'ph', 'p\'', 'q', 'qh', 'q\'', 'r', 's', 'sh', 't', 'th', 't\'', 'u', 'uu', 'w', 'y']

def grafemas_no_en_alfabet(words):
    for word in words:
        for i, letter in enumerate(word):
            #Continue  if letter is not a letter
            if not letter.isalpha():
                continue
            if letter.lower() not in alfabeto_quechua:
                #Chequear siguiente letra
                if i+1 >= len(word):
                    return False
                letter = letter + word[i+1]
                if letter.lower() not in alfabeto_quechua:
                    if i+2 >= len(word):
                        return False
                    #Chequear siguiente letra
                    letter = letter + word[i+2]
                    if letter.lower() not in alfabeto_quechua:
                        return False
    return True

#No funciona bien esta función, porque el detector de lenguaje no es muy bueno para palabras 
def oraciones_mucho_espaniol(words):
    spanish_words = 0
    for word in words:
        try:
            if detect(word) == 'es':
                spanish_words += 1
        except:
            pass
    return spanish_words/len(words) > 0.5

def oracion_mucho_espaniol_v2(words):
    palabras_encontradas = 0
    #Usar un diccionario de palabras en español
    for word in words:
        if word.lower() in spanish_words:
            palabras_encontradas += 1

    return palabras_encontradas/len(words) < 0.25


def oraciones_muy_cortas(words, min_length=3):
    return len(words) > min_length

def oraciones_muy_repititivas(words, threshold=0.4):
    unique_words = set(words)
    ratio = len(unique_words) / len(words)
    return ratio >= threshold

def palabras_muy_largas(words, threshold=40):
    for word in words:
        if len(word) > threshold:
            return False
    return True

def split_tokens(sentence):
    #Esta expresión busca cualquier secuencia de tres o más palabras que tengan cada una uno o dos caracteres de longitud, separadas por espacios.
    #En el paper se indica que es para detectar partes de una palabra que se rompieron durante la extracción de texto debido al formato del pdf.
    if re.search(r"(\b\w{1,2}\b\s){3,}", sentence):
        return False
    return True

def oraciones_con_matematica(sentence):
    if re.search(r"[\d+\-*/]+", sentence):
        return False
    return True

In [13]:
#a, aa, ch, chh, ch', ts, tr, h, i, ii, k, kh, k', l, ll, m, n, ñ, p, ph, p', q, qh, q', r, s, sh, t, th, t', u, uu, w, y
from nltk.tokenize import word_tokenize
# Reglas basadas en https://aclanthology.org/2020.lrec-1.356/
def rule_based_heuristic(sentence):
    words = word_tokenize(sentence)
    aux_solo_palabras = [word for word in words if word.isalpha()]
    
    valid = oraciones_muy_cortas(aux_solo_palabras) 
    valid = valid and oraciones_muy_repititivas(aux_solo_palabras)
    valid = valid and palabras_muy_largas(words)
    valid = valid and split_tokens(sentence)
    valid = valid and oraciones_con_matematica(sentence)
    valid = valid and oracion_mucho_espaniol_v2(aux_solo_palabras)

    return valid

In [11]:
def pipeline(documents):
    corpus=pd.DataFrame(columns=["document", "sentence"])
    for key in documents:
        text=get_all_text_from_pdf(key)
        sentences=sent_tokenize(text)
        #Oraciones antes de filtrar
        print("Oraciones antes de filtrar en documento ", key, ": ", len(sentences))
        filtered_sentences=[]
        for sentence in sentences:
            if rule_based_heuristic(sentence) and filtrar_oracion_espaniol(sentence):
                filtered_sentences.append(sentence)
        
        df=pd.DataFrame(filtered_sentences, columns=["sentence"])
        df["document"]=key
        #Quitar duplicados
        df.drop_duplicates(subset="sentence", inplace=True)
        print("Oraciones después de filtrar en documento ", key, ": ", len(df))
        corpus=pd.concat([corpus, df], ignore_index=True)
    print("Total de oraciones: ", len(corpus))
    corpus.drop_duplicates(subset="sentence", inplace=True)
    print("Total de oraciones únicas: ", len(corpus))
    return corpus

In [14]:
df_corpus=pipeline(documents)
df_corpus.head()

In [ ]:
df_corpus.loc[df_corpus["sentence"].apply(lambda x: len(x))<40]

In [ ]:
df_corpus.to_csv("data/corpus/avanceHarvy.csv", index=False)